**Подключение библиотек и скриптов**

# Курсовая работа "Предсказание стоимости дома"

для начала подгружаем все необходимые библиотеки и данные

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score as r2
from sklearn.model_selection import KFold, GridSearchCV
from datetime import datetime
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
matplotlib.rcParams.update({'font.size': 14})

TRAIN_DATASET_PATH = '/kaggle/input/real-estate-price-prediction-moscow/train.csv'
TEST_DATASET_PATH = '/kaggle/input/real-estate-price-prediction-moscow/test.csv'

train_df = pd.read_csv(TRAIN_DATASET_PATH)
test_df = pd.read_csv(TEST_DATASET_PATH)

после того, как мы подгрузили данные надо немного поразмашлять что они из себя представляют, какую ценность игграют и что вообще с ними можно сделать. Начнём с ID, ID вещь очень хорошая, но в моей курсовой абсолютно бессмысленная, так как у каждой квартиры он индивидуальный и никакой тайной зависимости цены - айдишника очевидно не будет. DistrictID также сам по себе важной информацией не является, но с помощью него мы можем узнать "плотность" домов в районе, что уже может быть довольно значимым признаком. Rooms, Square, LifeSquare, KitchenSquare очевидно важные признаки как и HouseYear. Этаж и количество этажей в доме не всегда влияют на цену из ходя из моего жизненного опыта, поэтому можно будет попробывать сделать предсказание без этих признаков. Про признаки экологии и социальных показателей местности мы не знаем ничего, поэтому о значимости этого признака мало что понимаем. Также можно предположить, что признаки экологии и наличии магазино с тц могут быть важными.

**Описание датасета**

* **Id** - идентификационный номер квартиры
* **DistrictId** - идентификационный номер района
* **Rooms** - количество комнат
* **Square** - площадь
* **LifeSquare** - жилая площадь
* **KitchenSquare** - площадь кухни
* **Floor** - этаж
* **HouseFloor** - количество этажей в доме
* **HouseYear** - год постройки дома
* **Ecology_1, Ecology_2, Ecology_3** - экологические показатели местности
* **Social_1, Social_2, Social_3** - социальные показатели местности
* **Healthcare_1, Helthcare_2** - показатели местности, связанные с охраной здоровья
* **Shops_1, Shops_2** - показатели, связанные с наличием магазинов, торговых центров
* **Price** - цена квартиры

In [ ]:
test_df.tail()

In [ ]:
print('Строк в трейне:', train_df.shape[0])
print('Строк в тесте', test_df.shape[0])

Приведём признаки ID и DistrictId к строковому типу, чтобы они не мешали модели

In [ ]:
train_df.dtypes

In [ ]:
train_df['Id'] = train_df['Id'].astype(str)
train_df['DistrictId'] = train_df['DistrictId'].astype(str)

## 1. EDA  <a class='anchor' id='eda'>
Теперь надо провести работу над данными, найти возможные выбросы,заполнить пропуски и сгенерировать новые признаки

**Целевая переменная**
начнём с изучения адекватности целевой переменной, из графика видно, что цена выглядит адекватно(очень дешёвых и очень дорогих домов мало, а домов средней цены много)

In [ ]:
plt.figure(figsize = (16, 8))

train_df['Price'].hist(bins=30)
plt.ylabel('Count')
plt.xlabel('Price')

plt.title('Target distribution')
plt.show()

Далее визуализируем наши данные, чтобы увидеть аномальные значения в некоторых признаках

In [ ]:
train_df.hist(figsize=(30, 30), bins=20, grid=False)

 Сразу видно, что потенциальные выбросы имеются в признаках:
-Rooms
-HouseFloor
-HouseYear
-KitchenSquare
-Square
-LifeSquare
Можно сделать такой вывод, потому что в данных есть значения, которые сильно отличаются от большинства значений


Начнём обрабатывать выбросы с признака Rooms
Посмотри сколько каких значений есть в этом признаке:

In [ ]:
train_df['Rooms'].value_counts()

Зачения больше шести комнат будем считать выбросами, таких квартир три
Также видим, что есть квартиры с нулём комнат, в вебинаре сказали, что это возможно студии и предложили заменить их на однокомнатную квартиру, но я думаю, что полезней будет оставить 0, потому что возможно с такой идеей ити данные и генирились, поэтому такое значение я оставлю

Пометим изменённые данные единичкой

In [ ]:
train_df['Rooms_outlier'] = 0
train_df.loc[ (train_df['Rooms'] > 6), 'Rooms_outlier'] = 1
train_df.loc[train_df['Rooms'] > 6, 'Rooms'] = train_df['Rooms'].median()

Квартиры с кол-вом комнат больше шести заменим на медиану

In [ ]:
train_df['Rooms'].value_counts()

Далее разберёмся с параметром KitchenSquare

In [ ]:
train_df['KitchenSquare'].value_counts()

Тут мы видим аномально большие значения, которые скорее всего выбросы, я заменю их на медианы, кухни с площадью 1 квадратный метр можно считать кухнями, которые совместили с другими комнатами, поэтому я не буду их заменять, возможно это важная информация

Пометим квартиры, которым собираемся менять площадь кухни и заменим её

In [ ]:
train_df['KitchenSquare_outlier'] = 0
train_df.loc[ (train_df['KitchenSquare'] > 18), 'Rooms_outlier'] = 1
train_df.loc[train_df['KitchenSquare'] > 18, 'KitchenSquare'] = train_df['KitchenSquare'].median()
train_df['KitchenSquare'].value_counts()

теперь посмотрим на такие параметры как этаж и кол-во этажей в доме, первый баг, который надо исправить, это что-то сделать с квартирами, у которых этаж больше, чем всего этажей в доме, я заменил их на максимальный этаж в доме, пометив эти квартиры. Далее я принял за выбросы дома с кол-вом этажей больше 50 и  0, 0 при этом заменил на 1, а больше 50 - на медиану.

In [ ]:
train_df['HouseFloor'].value_counts()

In [ ]:
train_df['HouseFloor']

In [ ]:
train_df['HouseFloor_outlier'] = 0
train_df.loc[train_df['HouseFloor'] == 0, 'HouseFloor_outlier'] = 1
train_df.loc[train_df['HouseFloor'] > 50, 'HouseFloor_outlier'] = 1
train_df.loc[train_df['HouseFloor'] > 50, 'HouseFloor_outlier'] = train_df['HouseFloor'].median()
train_df.loc[train_df['HouseFloor'] == 0, 'HouseFloor_outlier'] = train_df['HouseFloor'].median()
floor_outliers = train_df.loc[train_df['Floor'] > train_df['HouseFloor']].index
train_df.loc[floor_outliers, 'Floor'] = train_df.loc[floor_outliers, 'HouseFloor']

In [ ]:
(train_df['Floor'] > train_df['HouseFloor']).sum()

Теперь пофиксим анамалии в дате постройки дома, тут всё более менее хорошо, но некторые дома построены в будующем, тут я заменил дату постройки на текущий год

In [ ]:
train_df['HouseYear'].sort_values(ascending=False)

In [ ]:
current_year = datetime.now().year        
train_df['HouseYear_outlier'] = 0
train_df.loc[train_df['HouseYear'] > current_year, 'HouseYear_outlier'] = 1
train_df.loc[train_df['HouseYear'] > current_year, 'HouseYear'] = current_year

с площадью дома и сжилой площадью дома я поступил аналогично площади кухни, а именно принял за выбросы значения меньше 8ми и больше 300от, но заменил их не на медиану, а на квантили 25ти и 75ти процентов

In [ ]:
train_df['Square'].describe()

In [ ]:
train_df['Square_outlier'] = 0
train_df.loc[train_df['Square'] < 8, 'Square_outlier'] = 1
train_df.loc[train_df['Square'] > 300, 'Square_outlier'] = 1
train_df.loc[train_df['Square'] > 300, 'Square'] = train_df['Square'].quantile(.25)
train_df.loc[train_df['Square'] < 8, 'Square'] = train_df['Square'].quantile(.75)

In [ ]:
train_df['LifeSquare'].describe()

In [ ]:
train_df['LifeSquare_outlier'] = 0
train_df.loc[train_df['LifeSquare'] < 8, 'LifeSquare_outlier'] = 1
train_df.loc[train_df['LifeSquare'] > 300, 'LifeSquare_outlier'] = 1
train_df.loc[train_df['LifeSquare'] > 300, 'LifeSquare'] = train_df['LifeSquare'].quantile(.25)
train_df.loc[train_df['LifeSquare'] < 8, 'LifeSquare'] = train_df['LifeSquare'].quantile(.75)

Окей, если с выбросами всё, то надо обработать пропуски, для начало посмотрим сколько их

In [ ]:
train_df.isna().sum()

Видно, что пропуски в жилой площади и в признаке экологии, в признаке экологии почти половина строчек пустые, поэтому этот признак можно просто выкинуть, а жилую площадь я стандартно заменю на медиану

In [ ]:
train_df.drop('Healthcare_1', axis=1, inplace=True)
train_df.loc[train_df['LifeSquare'].isna() , 'LifeSquare_outlier'] = 1
train_df.loc[train_df['LifeSquare'].isna() , 'LifeSquare'] = train_df['LifeSquare'].median()

теперь, когда все данные в нормальном виде, можно сделать новые фичи

Для начала заменим бинарные признаки на числовые

In [ ]:
binary_to_numbers = {'A': 0, 'B': 1}
train_df['Ecology_2'] = train_df['Ecology_2'].replace(binary_to_numbers)
train_df['Ecology_3'] = train_df['Ecology_3'].replace(binary_to_numbers)
train_df['Shops_2'] = train_df['Shops_2'].replace(binary_to_numbers)

Сгенерим признак плотности домов районе(насколько много домов в этом районе)

In [ ]:
district_size = train_df['DistrictId'].value_counts().reset_index()\
                    .rename(columns={'index':'DistrictId', 'DistrictId':'DistrictSize'})
train_df = train_df.merge(district_size, on='DistrictId', how='left')
train_df.head()

In [ ]:
train_df = train_df.merge(district_size, on='DistrictId', how='left')
train_df.head()

создадим новый признак средняя цена на районе

In [ ]:
med_price_by_district = train_df.groupby(['DistrictId', 'Rooms'], as_index=False).agg({'Price':'median'})\
                            .rename(columns={'Price':'MedPriceByDistrict'})
train_df = train_df.merge(med_price_by_district, on=['DistrictId', 'Rooms'], how='left')

теперь все эти изменения занесём в класс

In [ ]:
class DataPreprocessing:
    """Подготовка исходных данных"""

    def __init__(self):
        """Параметры класса"""
        self.medians=None
        self.kitchen_square_quantile = None
        
    def fit(self, X):
        """Сохранение статистик"""       
        # Расчет медиан
        self.medians = X.median()
        self.kitchen_square_quantile = X['KitchenSquare'].quantile(.975)
    
    def transform(self, X):
        """Трансформация данных и генерация новых фич"""

        # Rooms
        X['Rooms_outlier'] = 0
        X.loc[ (X['Rooms'] > 6), 'Rooms_outlier'] = 1
        X.loc[X['Rooms'] > 6, 'Rooms'] = X['Rooms'].median()
        
        # KitchenSquare
        X['KitchenSquare_outlier'] = 0
        X.loc[ (X['KitchenSquare'] > 18), 'Rooms_outlier'] = 1
        X.loc[X['KitchenSquare'] > 18, 'KitchenSquare'] = X['KitchenSquare'].median()
        
        # HouseFloor, Floor
        X['HouseFloor_outlier'] = 0
        X.loc[X['HouseFloor'] == 0, 'HouseFloor_outlier'] = 1
        X.loc[X['HouseFloor'] > 50, 'HouseFloor_outlier'] = 1
        X.loc[X['HouseFloor'] > 50, 'HouseFloor_outlier'] = X['HouseFloor'].median()
        X.loc[X['HouseFloor'] == 0, 'HouseFloor_outlier'] = X['HouseFloor'].median()
        floor_outliers = X.loc[X['Floor'] > X['HouseFloor']].index
        X.loc[floor_outliers, 'Floor'] = X.loc[floor_outliers, 'HouseFloor']
        
        # HouseYear
        current_year = datetime.now().year        
        X['HouseYear_outlier'] = 0
        X.loc[X['HouseYear'] > current_year, 'HouseYear_outlier'] = 1
        X.loc[X['HouseYear'] > current_year, 'HouseYear'] = current_year

        X['Square_outlier'] = 0
        X.loc[X['Square'] < 8, 'Square_outlier'] = 1
        X.loc[X['Square'] > 300, 'Square_outlier'] = 1
        X.loc[X['Square'] > 300, 'Square'] = X['Square'].quantile(.25)
        X.loc[X['Square'] < 8, 'Square'] = X['Square'].quantile(.75)

        X['LifeSquare_outlier'] = 0
        X.loc[X['LifeSquare'] < 8, 'LifeSquare_outlier'] = 1
        X.loc[X['LifeSquare'] > 300, 'LifeSquare_outlier'] = 1
        X.loc[X['LifeSquare'] > 300, 'LifeSquare'] = X['LifeSquare'].quantile(.25)
        X.loc[X['LifeSquare'] < 8, 'LifeSquare'] = X['LifeSquare'].quantile(.75)

        X.drop('Healthcare_1', axis=1, inplace=True)
        X.loc[X['LifeSquare'].isna() , 'LifeSquare_outlier'] = 1
        X.loc[X['LifeSquare'].isna() , 'LifeSquare'] = X['LifeSquare'].median()

        binary_to_numbers = {'A': 0, 'B': 1}
        X['Ecology_2'] = X['Ecology_2'].replace(binary_to_numbers)
        X['Ecology_3'] = X['Ecology_3'].replace(binary_to_numbers)
        X['Shops_2'] = X['Shops_2'].replace(binary_to_numbers)

        district_size = X['DistrictId'].value_counts().reset_index()\
                    .rename(columns={'index':'DistrictId', 'DistrictId':'DistrictSize'})
        X = X.merge(district_size, on='DistrictId', how='left')
        
        
        X.fillna(self.medians, inplace=True)
        
        return X

### 5. Отбор признаков  <a class='anchor' id='feature_selection'>

In [ ]:
train_df.columns.tolist()

In [ ]:
feature_names = ['Rooms', 'Square', 'LifeSquare', 'KitchenSquare', 'Floor', 'HouseFloor', 'HouseYear',
                 'Ecology_1', 'Ecology_2', 'Ecology_3', 'Social_1', 'Social_2', 'Social_3',
                 'Helthcare_2', 'Shops_1', 'Shops_2']

new_feature_names = ['Rooms_outlier', 'HouseFloor_outlier', 'HouseYear_outlier',  'DistrictSize']

target_name = 'Price'

разобьем анши данные на тренировочные и валидационные

In [ ]:
train_df = pd.read_csv(TRAIN_DATASET_PATH)
test_df = pd.read_csv(TEST_DATASET_PATH)

X = train_df.drop(columns=target_name)
y = train_df[target_name]
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.33, shuffle=True, random_state=21)

трансформируем наши данные

In [ ]:
preprocessor = DataPreprocessing()
preprocessor.fit(X_train)

X_train = preprocessor.transform(X_train)
X_valid = preprocessor.transform(X_valid)
test_df = preprocessor.transform(test_df)

X_train.shape, X_valid.shape, test_df.shape

In [ ]:
X_train = X_train[feature_names + new_feature_names]
X_valid = X_valid[feature_names + new_feature_names]
test_df = test_df[feature_names + new_feature_names]

In [ ]:
X_train.isna().sum().sum(), X_valid.isna().sum().sum(), test_df.isna().sum().sum()

### 7. Построение модели  <a class='anchor' id='modeling'>

**Обучение**

In [ ]:
def evaluate_preds(train_true_values, train_pred_values, test_true_values, test_pred_values):
    print("Train R2:\t" + str(round(r2(train_true_values, train_pred_values), 3)))
    print("Test R2:\t" + str(round(r2(test_true_values, test_pred_values), 3)))
    
    plt.figure(figsize=(18,10))
    
    plt.subplot(121)
    sns.scatterplot(x=train_pred_values, y=train_true_values)
    plt.xlabel('Predicted values')
    plt.ylabel('True values')
    plt.title('Train sample prediction')
    
    plt.subplot(122)
    sns.scatterplot(x=test_pred_values, y=test_true_values)
    plt.xlabel('Predicted values')
    plt.ylabel('True values')
    plt.title('Test sample prediction')

    plt.show()

Перебрав разные модели, я пришёл к выводу, что BoostingRegressor лучшее решение при таких параметрах

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
gb_model = GradientBoostingRegressor(random_state=21, criterion='mse', max_depth=3, min_samples_leaf=25, n_estimators=200)
gb_model.fit(X_train, y_train)
y_train_preds = gb_model.predict(X_train)
y_test_preds = gb_model.predict(X_valid)

evaluate_preds(y_train, y_train_preds, y_valid, y_test_preds)

### 8. Прогнозирование на тестовом датасете  <a class='anchor' id='prediction'>

1. Выполнить для тестового датасета те же этапы обработки и постронияния признаков
2. Не потерять и не перемешать индексы от примеров при построении прогнозов
3. Прогнозы должны быть для все примеров из тестового датасета (для всех строк)

In [ ]:
test_df.shape

In [ ]:
test_df

In [ ]:
submit = pd.read_csv('/kaggle/input/real-estate-price-prediction-moscow/sample_submission.csv')
submit.head()

In [ ]:
predictions = gb_model.predict(test_df)
predictions

In [ ]:
submit['Price'] = predictions
submit.head()

In [ ]:
submit.to_csv('rf_submit.csv', index=False)